In [33]:
#Import libraries
import googlemaps
import requests
from itertools import tee
import time
import pandas as pd
apikey= 'Type API here'

In [2]:
#Instantiate google maps 
gmaps = googlemaps.Client(key=apikey)


In [3]:
#Create function to turn data pulled from API into a dataframe
def create_df(places_result):
    #create empty list to hold informaion for each column
    geometry = []
    name = []
    vicinity = []
    types = []
    #Create a for loop to iterate over dictionary and append to lists.
    for place in places_result['results']:
        geometry.append(place['geometry'])
        name.append(place['name'])
        vicinity.append(place['vicinity'])
        types.append(place['types'])
    #Turn lists into a dataframe
    df = pd.DataFrame([geometry,name,vicinity,types])
    df = df.T
    df.columns = ['geometry','name','vicinity','types']
    return df

In [4]:
#Create function to pull from Google maps API and return a dataframe.
#Inputs will be the loc('latitude,longitude'), radius to search, and place type.
def find_nearby(loc,rad,place_types):
    #Pull data from Google API and use create_df to create dataframe.
    places_result = gmaps.places_nearby(location = loc,radius=rad, type=place_types,page_token=None)
    df= create_df(places_result)
    #Call next page and store as after
    after=places_result.get('next_page_token',None)
    time.sleep(3)
    #repeat this process until there are no more pages to pull from.
    while after !=None:
        places_result = gmaps.places_nearby(location = loc,radius=rad, type=place_types,page_token=after)
        df = df.append(create_df(places_result))
        after=places_result.get('next_page_token',None)
        time.sleep(3) # Set a 3 second delay between requests.
    df.reset_index(drop=True,inplace=True)
    return df
    

In [34]:
df = find_nearby('29.9511,-90.0715',20000,'police')

In [35]:
df.head()

,geometry,name,vicinity,types
0,"{'location': {'lat': 29.951772, 'lng': -90.076...",Orleans Parish Sheriffs Office,"421 Loyola Avenue, New Orleans","[police, point_of_interest, establishment]"
1,"{'location': {'lat': 29.9553436, 'lng': -90.06...",New Orleans Police Department,"334 Royal Street, New Orleans","[police, point_of_interest, establishment]"
2,"{'location': {'lat': 29.9034746, 'lng': -90.07...",Jefferson Parish Sheriff's Office,"725 Maple Avenue, Harvey","[police, local_government_office, point_of_int..."
3,"{'location': {'lat': 29.9600404, 'lng': -90.10...",New Orleans Police Department - Second Distric...,"3401 Broadway Street, New Orleans","[police, point_of_interest, establishment]"
4,"{'location': {'lat': 29.8920146, 'lng': -89.97...",Plaquemine Parish Narcotics,"505 F Edward Hebert Boulevard, Belle Chasse","[police, local_government_office, point_of_int..."


In [36]:
df.tail()

,geometry,name,vicinity,types
55,"{'location': {'lat': 29.942584, 'lng': -90.117...",Tulane University Police Department,"2650 Ben Weiner Drive, New Orleans","[police, university, point_of_interest, establ..."
56,"{'location': {'lat': 29.953974, 'lng': -90.074...",Tulane University Police Department,"1201 Tulane Avenue, New Orleans","[police, point_of_interest, establishment]"
57,"{'location': {'lat': 29.9446748, 'lng': -90.06...",Habor Police Office,"The Outlet Collection at Riverwalk, at Riverwa...","[police, point_of_interest, establishment]"
58,"{'location': {'lat': 29.9602049, 'lng': -90.09...",New Orleans Police Department - Headquarters,"715 South Broad Avenue, New Orleans","[police, point_of_interest, establishment]"
59,"{'location': {'lat': 29.9655568, 'lng': -90.08...",new orleans independent police monitor,"2714 Canal Street, New Orleans","[police, point_of_interest, establishment]"


In [37]:
df.shape

(60, 4)

In [38]:
#Loop through the geometry column of dataframe and create new latitude and longitude columns.
lat_list=[]
lng_list=[]
for place in df['geometry']:
    lat_list.append(place['location']['lat'])
    lng_list.append(place['location']['lng'])

place_df = pd.DataFrame([lat_list,lng_list]).T
place_df.columns=['lat','lng']

#Merge latitude and longitude to dataframe.
df= df.merge(place_df,left_index=True, right_index=True)

In [39]:
df

,geometry,name,vicinity,types,lat,lng
0,"{'location': {'lat': 29.951772, 'lng': -90.076...",Orleans Parish Sheriffs Office,"421 Loyola Avenue, New Orleans","[police, point_of_interest, establishment]",29.951772,-90.076146
1,"{'location': {'lat': 29.9553436, 'lng': -90.06...",New Orleans Police Department,"334 Royal Street, New Orleans","[police, point_of_interest, establishment]",29.955344,-90.066900
2,"{'location': {'lat': 29.9034746, 'lng': -90.07...",Jefferson Parish Sheriff's Office,"725 Maple Avenue, Harvey","[police, local_government_office, point_of_int...",29.903475,-90.071188
3,"{'location': {'lat': 29.9600404, 'lng': -90.10...",New Orleans Police Department - Second Distric...,"3401 Broadway Street, New Orleans","[police, point_of_interest, establishment]",29.960040,-90.108834
4,"{'location': {'lat': 29.8920146, 'lng': -89.97...",Plaquemine Parish Narcotics,"505 F Edward Hebert Boulevard, Belle Chasse","[police, local_government_office, point_of_int...",29.892015,-89.976610
5,"{'location': {'lat': 30.01163269999999, 'lng':...",Sheriff's Office-Property Tax,"4116 Hessmer Avenue, Metairie","[police, local_government_office, point_of_int...",30.011633,-90.163725
6,"{'location': {'lat': 29.8896617, 'lng': -90.11...",Jefferson Parish Sheriff,"1425 Walkertown Way, Marrero","[police, local_government_office, point_of_int...",29.889662,-90.111752
7,"{'location': {'lat': 29.9412793, 'lng': -90.08...",New Orleans Police Department,"1930 Martin Luther King Junior Boulevard, New ...","[police, point_of_interest, establishment]",29.941279,-90.080115
8,"{'location': {'lat': 29.90622549999999, 'lng':...",Jefferson Parish Sheriff's Office,"1233 Westbank Expressway, Harvey","[police, point_of_interest, establishment]",29.906225,-90.064203
9,"{'location': {'lat': 29.9484696, 'lng': -90.06...",US Marshals Services,"500 Poydras Street, New Orleans","[police, point_of_interest, establishment]",29.948470,-90.068984


## Stop

In [40]:
#export to csv.
df.to_csv('./datasets/police.csv', index = False)

In [45]:
firestaton_df = find_nearby('29.9511,-90.0715',8000,'fire_station')

In [46]:
firestaton_df.reset_index(drop=True, inplace= True)

In [47]:
firestaton_df.head()

,geometry,name,vicinity,types
0,"{'location': {'lat': 29.95414740000001, 'lng':...",New Orleans Fire Department,"317 Decatur Street, New Orleans","[fire_station, point_of_interest, establishment]"
1,"{'location': {'lat': 29.95720500000001, 'lng':...",Fire Station 14,New Orleans,"[fire_station, point_of_interest, establishment]"
2,"{'location': {'lat': 29.94844839999999, 'lng':...",New Orleans Fire Engine 20,"425 Opelousas Avenue, New Orleans","[fire_station, point_of_interest, establishment]"
3,"{'location': {'lat': 29.9478144, 'lng': -90.07...",NOFD Fire Station Squad 2/Rescue 2,"801 Girod Street, New Orleans","[fire_station, point_of_interest, establishment]"
4,"{'location': {'lat': 29.94158549999999, 'lng':...",Cease Fire,"1417 Oretha Castle Haley Boulevard, New Orleans","[fire_station, point_of_interest, establishment]"


In [48]:
firestaton_df.shape

(56, 4)

In [56]:
#Loop through the geometry column of dataframe and create new latitude and longitude columns.
lat_list=[]
lng_list=[]
for place in firestaton_df['geometry']:
    lat_list.append(place['location']['lat'])
    lng_list.append(place['location']['lng'])

place_df = pd.DataFrame([lat_list,lng_list]).T
place_df.columns=['lat','lng']

#Merge latitude and longitude to dataframe.
firestaton_df= firestaton_df.merge(place_df,left_index=True, right_index=True)

In [64]:
firestaton_df.to_csv('./datasets/firestation.csv', index = False)

In [58]:
firestaton_df

,geometry,name,vicinity,types,lat,lng
0,"{'location': {'lat': 29.95414740000001, 'lng':...",New Orleans Fire Department,"317 Decatur Street, New Orleans","[fire_station, point_of_interest, establishment]",29.954147,-90.065442
1,"{'location': {'lat': 29.95720500000001, 'lng':...",Fire Station 14,New Orleans,"[fire_station, point_of_interest, establishment]",29.957205,-90.077615
2,"{'location': {'lat': 29.94844839999999, 'lng':...",New Orleans Fire Engine 20,"425 Opelousas Avenue, New Orleans","[fire_station, point_of_interest, establishment]",29.948448,-90.051754
3,"{'location': {'lat': 29.9478144, 'lng': -90.07...",NOFD Fire Station Squad 2/Rescue 2,"801 Girod Street, New Orleans","[fire_station, point_of_interest, establishment]",29.947814,-90.072802
4,"{'location': {'lat': 29.94158549999999, 'lng':...",Cease Fire,"1417 Oretha Castle Haley Boulevard, New Orleans","[fire_station, point_of_interest, establishment]",29.941585,-90.079054
5,"{'location': {'lat': 29.96275099999999, 'lng':...",NOFD Station 7,"1441 South Peters Street, New Orleans","[fire_station, point_of_interest, establishment]",29.962751,-90.071415
6,"{'location': {'lat': 29.9414887, 'lng': -90.08...",NOFD Fire Station Engine 16,"2000 Martin Luther King Junior Boulevard, New ...","[fire_station, point_of_interest, establishment]",29.941489,-90.080522
7,"{'location': {'lat': 29.962158, 'lng': -90.057...",New Orleans Fire Department Station 9,"449 Esplanade Avenue, New Orleans","[fire_station, point_of_interest, establishment]",29.962158,-90.057771
8,"{'location': {'lat': 29.9663987, 'lng': -90.09...",NOFD Fire Station Engine 26/Ladder 9,"436 South Jefferson Davis Parkway, New Orleans","[fire_station, point_of_interest, establishment]",29.966399,-90.096658
9,"{'location': {'lat': 29.9249655, 'lng': -90.08...",Fire Station 1,"2924 Magazine Street, New Orleans","[fire_station, point_of_interest, establishment]",29.924965,-90.084176


In [59]:
airport_df = find_nearby('29.9511,-90.0715',8000,'airport')

In [60]:
airport_df.shape

(16, 4)

In [61]:
#Loop through the geometry column of dataframe and create new latitude and longitude columns.
lat_list=[]
lng_list=[]
for place in airport_df['geometry']:
    lat_list.append(place['location']['lat'])
    lng_list.append(place['location']['lng'])

place_df = pd.DataFrame([lat_list,lng_list]).T
place_df.columns=['lat','lng']

#Merge latitude and longitude to dataframe.
airport_df= airport_df.merge(place_df,left_index=True, right_index=True)

In [63]:
airport_df.to_csv('./datasets/airports.csv', index = False)

In [66]:
df = find_nearby('29.9511,-90.0715',8000,'bus_station')

In [67]:
#Loop through the geometry column of dataframe and create new latitude and longitude columns.
lat_list=[]
lng_list=[]
for place in df['geometry']:
    lat_list.append(place['location']['lat'])
    lng_list.append(place['location']['lng'])

place_df = pd.DataFrame([lat_list,lng_list]).T
place_df.columns=['lat','lng']

#Merge latitude and longitude to dataframe.
df= df.merge(place_df,left_index=True, right_index=True)

In [68]:
df.head()

,geometry,name,vicinity,types,lat,lng
0,"{'location': {'lat': 29.9198326, 'lng': -90.04...",Gretna Bus Terminal,Gretna,"[bus_station, transit_station, point_of_intere...",29.919833,-90.043902
1,"{'location': {'lat': 29.949982, 'lng': -90.071...",Poydras at Carondelet,United States,"[bus_station, transit_station, point_of_intere...",29.949982,-90.071649
2,"{'location': {'lat': 29.950106, 'lng': -90.070...",New Orln Charles Poydras,United States,"[bus_station, transit_station, point_of_intere...",29.950106,-90.070288
3,"{'location': {'lat': 29.950269, 'lng': -90.072...",Poydras at Baronne,United States,"[bus_station, transit_station, point_of_intere...",29.950269,-90.072919
4,"{'location': {'lat': 29.950322, 'lng': -90.073...",Baronne at Poydras,United States,"[bus_station, transit_station, point_of_intere...",29.950322,-90.073093


In [69]:
df.to_csv('./datasets/bus.csv', index = False)